<a href="https://colab.research.google.com/github/PrishaAbrol/deepfake/blob/main/deep_fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -U -q evaluate transformers datasets accelerate torch torchvision

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from google.colab import drive

# Attempt to mount Google Drive
try:
    drive.mount('/content/drive')
except Exception as e:
    # If mounting fails, print the error message and suggest solutions
    print(f"Error mounting Google Drive: {e}")
    print("Possible solutions:")
    print("- Restart the runtime and reauthenticate.")
    print("- Ensure you are logged into the correct Google account.")
    print("- Clear your browser's cache and cookies.")

# Define paths to each folder
base_dir = '/content/drive/MyDrive/hack'  # Replace 'hack' with the actual folder name if different
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')
val_dir = os.path.join(base_dir, 'Validation')

# Verify if the directories exist
for dir_path in [train_dir, test_dir, val_dir]:
    if not os.path.exists(dir_path):
        print(f"Error: Directory '{dir_path}' not found. Please create the directory or adjust the path.")

# Function to load images and labels from a directory
def load_images_from_folder(folder):
    images = []
    labels = []
    import os

# Function to load images and labels from a directory
def load_images_from_folder(folder):
    images = []
    labels = []

    # Check if the folder exists and is a directory
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Error: Directory '{folder}' not found.")

    if not os.path.isdir(folder):
        raise NotADirectoryError(f"Error: '{folder}' is not a directory.")

    # Iterate through each class folder
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)

        # Ensure the current item is a directory
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                try:
                    img = Image.open(img_path).resize((224, 224))  # Resize images
                    img_array = np.array(img) / 255.0  # Normalize the image
                    images.append(img_array)
                    labels.append(class_folder)  # Use folder name as the label
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels)



# Load images and labels from each folder
train_images, train_labels = load_images_from_folder(train_dir)
test_images, test_labels = load_images_from_folder(test_dir)
val_images, val_labels = load_images_from_folder(val_dir)

# Print some info
print("Train set:", train_images.shape, train_labels.shape)
print("Test set:", test_images.shape, test_labels.shape)
print("Validation set:", val_images.shape, val_labels.shape)

# Display a few images with their labels
for i in range(5):  # Show 5 images
    plt.imshow(train_images[i])
    plt.title(f"Label: {train_labels[i]}")
    plt.axis('off')
    plt.show()

# Import necessary modules
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings during execution
import gc
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

# Handle truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Create a pandas dataframe for images and labels
from pathlib import Path
file_names = []
labels = []
for file in sorted((Path('/content/drive/MyDrive/hack/train').glob('*/*.*'))):
    label = str(file).split('/')[-2]
    labels.append(label)
    file_names.append(str(file))

df = pd.DataFrame.from_dict({"image": file_names, "label": labels})
print(df['label'].unique())  # Print unique labels

# Handle class imbalance using RandomOverSampler
from imblearn.over_sampling import RandomOverSampler
y = df['label'].to_numpy()
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df_resampled, y_resampled = ros.fit_resample(df, y)
df_resampled['label'] = y_resampled
gc.collect()

# Convert pandas DataFrame into Dataset object
from datasets import Dataset, Image, ClassLabel
dataset = Dataset.from_pandas(df_resampled).cast_column("image", Image())

# Set labels
labels_list = ['Real', 'Fake']
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for i, label in enumerate(labels_list)}
ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)

# Define data transformations
from torchvision.transforms import Compose, Resize, RandomRotation, RandomAdjustSharpness, ToTensor, Normalize
from torch import tensor

model_str = "dima806/deepfake_vs_real_image_detection"
from transformers import ViTImageProcessor
processor = ViTImageProcessor.from_pretrained(model_str)
image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose([Resize((size, size)), RandomRotation(90), RandomAdjustSharpness(2), ToTensor(), normalize])
_val_transforms = Compose([Resize((size, size)), ToTensor(), normalize])

# Apply transformations to datasets
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset['train']
test_data = dataset['test']
train_data.set_transform(lambda examples: {'pixel_values': [_train_transforms(image.convert("RGB")) for image in examples['image']]})
test_data.set_transform(lambda examples: {'pixel_values': [_val_transforms(image.convert("RGB")) for image in examples['image']]})

# Define model and training arguments
from transformers import ViTForImageClassification, Trainer, TrainingArguments

model = ViTForImageClassification.from_pretrained(model_str, num_labels=len(labels_list))
model.config.id2label = id2label
model.config.label2id = label2id

args = TrainingArguments(
    output_dir="deepfake_vs_real_image_detection",
    evaluation_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.02,
    save_strategy='epoch',
    load_best_model_at_end=True
)

# Create Trainer object
import torch
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=lambda examples: {
        'pixel_values': torch.stack([example["pixel_values"] for example in examples]),
        'labels': torch.tensor([example['label'] for example in examples])
    },
    compute_metrics=lambda eval_pred: {
        "accuracy": accuracy_score(eval_pred.label_ids, eval_pred.predictions.argmax(1))
    }
)

# Train and evaluate model
trainer.train()
trainer.evaluate()

# Confusion matrix plot function
y_true = trainer.predict(test_data).label_ids
y_pred = trainer.predict(test_data).predictions.argmax(1)

def plot_confusion_matrix(cm, classes, title='Confusion Matrix'):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    plt.xticks(np.arange(len(classes)), classes, rotation=45)
    plt.yticks(np.arange(len(classes)), classes)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, classes=labels_list)


Error mounting Google Drive: Error: credential propagation was unsuccessful
Possible solutions:
- Restart the runtime and reauthenticate.
- Ensure you are logged into the correct Google account.
- Clear your browser's cache and cookies.
Error: Directory '/content/drive/MyDrive/hack/Train' not found. Please create the directory or adjust the path.
Error: Directory '/content/drive/MyDrive/hack/Test' not found. Please create the directory or adjust the path.
Error: Directory '/content/drive/MyDrive/hack/Validation' not found. Please create the directory or adjust the path.


FileNotFoundError: Error: Directory '/content/drive/MyDrive/hack/Train' not found.